In [1]:
import tensorflow as tf
slim = tf.contrib.slim
from PIL import Image
from inception_v4 import *
import numpy as np
import pickle
import random
import os
import time

In [2]:
model_directory1 = "inception"
model_directory2 = "SavedModels"
data_directory = "101_ObjectCategories/"

checkpoint_file = os.getcwd() + '\inception_v4.ckpt'
sample_images = 'cropped_panda.jpg'
sample_labels = [0, 1, 0, 0, 0]
num_classes = 102

In [3]:
decode_jpeg_data = tf.placeholder(tf.string)
decode_jpeg = tf.image.decode_jpeg(decode_jpeg_data, channels=3)
if decode_jpeg.dtype != tf.float32:
    decode_jpeg = tf.image.convert_image_dtype(decode_jpeg, dtype=tf.float32)
image = tf.expand_dims(decode_jpeg, 0)
image = tf.image.resize_bilinear(image, [299,299], align_corners=False)
scaled_input_tensor = tf.subtract(image, 0.5)
scaled_input_tensor = tf.multiply(scaled_input_tensor, 2.0)

#Load the model
sess = tf.Session()
arg_scope = inception_v4_arg_scope()

#exclude = ['InceptionResnetV2/Logits', 'InceptionResnetV2/AuxLogits']
with slim.arg_scope(arg_scope):
      logits, end_points = inception_v4(scaled_input_tensor, is_training=False)
saver = tf.train.Saver()
saver.restore(sess, checkpoint_file)

In [5]:
image_data = tf.gfile.FastGFile(sample_images, 'rb').read()
predictions, logits = sess.run([end_points['Predictions'], end_points['Logits']], feed_dict = {decode_jpeg_data: image_data})

In [8]:
np.argmax(predictions)

389

In [4]:
def getBottleneckValues(image_path):
    image_data = tf.gfile.FastGFile(image_path, 'rb').read()
    #image_data = image_data.reshape(-1,299,299,3)
    bottleneck_values = sess.run(end_points['PreLogitsFlatten'], feed_dict={decode_jpeg_data: image_data})
    return np.squeeze(bottleneck_values)

In [5]:
def bottleneckCache(cache_path, image_paths=None):
    if os.path.exists(cache_path):
    # Load the cached data from the file.
        with open(cache_path, mode='rb') as file:
            bottleneck_values = pickle.load(file)

        print("Data loaded: " + cache_path)
    else:
        # No cache file. Make it.
        num_images = len(image_paths)
        # allocate memory in advance
        bottleneck_values = [None] * num_images

        # For each input image.
        for i in range(num_images):
            bottleneck_values[i] = getBottleneckValues(image_path=os.path.abspath(data_directory+image_paths[i]))
        
        with open(cache_path, mode='wb') as file:
            pickle.dump(np.array(bottleneck_values), file)

        print("Data saved: " + cache_path)

    return bottleneck_values

In [8]:
def getFiles(dire):
    l=[]
    g =[]
    for root, dirs, files in os.walk(dire):
        if dirs:
            names = dirs
        if files:
            r= []
            for name in files:
                r.append(root.split(os.path.sep)[-1] + "/" + name)
                l.append((root.split(os.path.sep)[-1]))
            g.append(r[:])
    return g, names

In [9]:
output, names = getFiles(os.path.abspath(data_directory))

In [10]:
def splitClasses(images, training_amount):
    #expects a list of lists, where the outer represents the class, and inner is the images of the class
    training_data = []
    testing_data = []
    for i in range(len(images)):
        random.shuffle(images[i])
        training_data.append(images[i][:training_amount])
        testing_data.append(images[i][training_amount:])
    return training_data, testing_data

In [11]:
import time
start = time.time()
dataset_folder_name = '256Dataset-ResNet/'
bottleneck_values = []
if not os.path.exists(dataset_folder_name):
    os.mkdir(dataset_folder_name)
for i in range(len(names)):
    bottleneck_values.append(bottleneckCache(cache_path=os.path.abspath(dataset_folder_name + names[i]+ '.pkl'),image_paths=output[i]))
end = time.time()

Data saved: C:\Users\Seth\Tutorials\CNNTutorials\5. TransferLearning_Tutorial\256Dataset-ResNet\001.ak47.pkl
Data saved: C:\Users\Seth\Tutorials\CNNTutorials\5. TransferLearning_Tutorial\256Dataset-ResNet\002.american-flag.pkl
Data saved: C:\Users\Seth\Tutorials\CNNTutorials\5. TransferLearning_Tutorial\256Dataset-ResNet\003.backpack.pkl
Data saved: C:\Users\Seth\Tutorials\CNNTutorials\5. TransferLearning_Tutorial\256Dataset-ResNet\004.baseball-bat.pkl
Data saved: C:\Users\Seth\Tutorials\CNNTutorials\5. TransferLearning_Tutorial\256Dataset-ResNet\005.baseball-glove.pkl
Data saved: C:\Users\Seth\Tutorials\CNNTutorials\5. TransferLearning_Tutorial\256Dataset-ResNet\006.basketball-hoop.pkl
Data saved: C:\Users\Seth\Tutorials\CNNTutorials\5. TransferLearning_Tutorial\256Dataset-ResNet\007.bat.pkl
Data saved: C:\Users\Seth\Tutorials\CNNTutorials\5. TransferLearning_Tutorial\256Dataset-ResNet\008.bathtub.pkl
Data saved: C:\Users\Seth\Tutorials\CNNTutorials\5. TransferLearning_Tutorial\256Dat

In [24]:
training_data2, testing_data2 = splitClasses(bottleneck_values, 60)

In [25]:
#This code takes the 2d lists and flattens them out into 1d. It also generates the label lists
training_data = []
training_labels = []
testing_data = []
testing_labels = []
for i in range(len(training_data2[:])):
    for j in range (len(training_data2[i])):
        training_labels.append(names[i])
        
training_data = [val for sublist in training_data2 for val in sublist]

for i in range(len(testing_data2[:])):
    for j in range (len(testing_data2[i])):
        testing_labels.append(names[i])
        
testing_data = [val for sublist in testing_data2 for val in sublist]

In [26]:
# zip data
zipped_values = list(zip(training_labels, training_data))

#shuffle the data
random.shuffle(zipped_values)

#unzip new shuffled data
training_labels, training_data = zip(*zipped_values)

# zip data
zipped_values = list(zip(testing_labels, testing_data))

#shuffle the data
random.shuffle(zipped_values)

#unzip new shuffled data
testing_labels, testing_data = zip(*zipped_values)

In [27]:
# Converts label name to a one hot vector
# eg: if you had classes cat and dog, and an image was the dog, it's label would be [0, 1]
mydict = {}
i=0
for name in names:
    mydict[name] = i
    i += 1
def labelNamesToValues(labels):
    list_of_labels = []
    one_hot_labels = [0] * len(mydict.values())
    for label in labels:
        new_list = one_hot_labels[:]
        new_list[mydict[label]] = 1
        list_of_labels.append(new_list)
        
    return list_of_labels

In [28]:
train_labels = labelNamesToValues(training_labels)
test_labels = labelNamesToValues(testing_labels)

# Create new Classification Layer

In [19]:
bottleneck_length = 1536

graph = tf.Graph()
with graph.as_default():
    
    x_input = tf.placeholder(tf.float32, shape=[None, bottleneck_length])
    y_output = tf.placeholder(tf.float32, shape=[None, num_classes])
    y_output_class = tf.argmax(y_output, dimension=1)
    
    num_epochs=500
    batch_size = 128
    
    global_step = tf.Variable(0, trainable=False)
    #learning rate
    learning_rate = tf.train.exponential_decay(.01, global_step, num_epochs, 0.90, staircase = True)
    
    with tf.variable_scope('newfc'):
        weights = tf.get_variable('weights', shape=[bottleneck_length, num_classes],
                                 initializer = tf.contrib.layers.xavier_initializer())
        biases = tf.get_variable('biases', shape=[num_classes], initializer=tf.constant_initializer(0))

        matrix_multiply = tf.matmul(x_input, weights)

        y_out = tf.nn.bias_add(matrix_multiply, biases)
        
    y_class = tf.nn.softmax(logits=y_out)
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=y_output, logits=y_out)
    cross_entropy_mean = tf.reduce_mean(cross_entropy)

    correct_prediction = tf.equal(tf.argmax(y_out,1), tf.argmax(y_output,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    #optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy_mean)
    optimizer = tf.train.MomentumOptimizer(learning_rate, 0.9).minimize(cross_entropy_mean)

In [29]:
model_name = "ResNet-Caltech256-60-1"
model_filename = model_name + "1000epochsModel.ckpt"
model_directory2 = os.getcwd() + "/SavedModels/" + model_name
model_path = model_directory2 + "/" + model_filename

In [30]:
def batch(x_data,y_data=None, batches=1):
    length = len(x_data)
    
    # return iterations
    for index in range(0, length, batches):
        if y_data is None:
            yield x_data[index:min(index + batches, length)]
        else:
            yield x_data[index:min(index + batches, length)], y_data[index:min(index + batches, length)]

In [31]:
def getTime(seconds):
    m, s = divmod(seconds, 60)
    h, m = divmod(m, 60)
    
    return h, m, s

In [ ]:
start = time.time()
train_accuracy_list = []
valid_accuracy_list = []
train_loss_list = []
valid_loss_list = []
time_list = []
epoch_list = []

print("TRAINING ")

with tf.Session(graph = graph) as session:
    session.run(tf.global_variables_initializer())
    saver = tf.train.Saver()
    #if os.path.exists(model_directory2):
    #    print("Loading model...")
    #    load_path = saver.restore(session, model_path)
    for current_epoch in range(1000):
        for x, y in batch(training_data, train_labels, batch_size):
            feed_dict = {x_input: x, y_output: y}
            # training and optimizing
            session.run([optimizer], feed_dict = feed_dict)

        train_accuracy = accuracy.eval(feed_dict={x_input:training_data, y_output: train_labels})
        train_accuracy_list.append(train_accuracy)
        valid_accuracy = accuracy.eval(feed_dict={x_input:testing_data, y_output: test_labels})
        valid_accuracy_list.append(valid_accuracy)
        train_loss = cross_entropy_mean.eval(feed_dict={x_input:training_data, y_output: train_labels})
        train_loss_list.append(train_loss)
        valid_loss = cross_entropy_mean.eval(feed_dict={x_input:testing_data, y_output: test_labels})
        valid_loss_list.append(valid_loss)
        
        current_time = time.time() - start
        hours, minutes, seconds = getTime(current_time)
        print("Epoch[%d]" % current_epoch + "%d" % hours + ":%2d" % minutes + ":%2d " % seconds + "%f " % train_accuracy + " %f" % valid_accuracy + " %f " % train_loss + " %f" % valid_loss)
        time_list.append(current_time)
        epoch_list.append(current_epoch)
        # Evaluate on test dataset.
    if not os.path.exists(model_directory2):
        os.mkdir(model_directory2)
    save_path = saver.save(session, model_path)

TRAINING 
Epoch[0]0: 0: 1 0.706225  0.676302 3.531881  3.558266
Epoch[1]0: 0: 3 0.803112  0.756568 1.966991  2.064741
Epoch[2]0: 0: 5 0.829701  0.784421 1.275335  1.416380
Epoch[3]0: 0: 7 0.843774  0.798907 0.984466  1.147618
Epoch[4]0: 0: 8 0.852464  0.807402 0.832376  1.010339
Epoch[5]0: 0:10 0.861154  0.813394 0.737034  0.927306
Epoch[6]0: 0:12 0.867639  0.817673 0.669811  0.871268
Epoch[7]0: 0:14 0.873216  0.820571 0.618627  0.830577
Epoch[8]0: 0:16 0.879377  0.822875 0.577564  0.799489
Epoch[9]0: 0:17 0.884176  0.825773 0.543381  0.774851
Epoch[10]0: 0:19 0.888975  0.827419 0.514149  0.754782
Epoch[11]0: 0:21 0.893839  0.829460 0.488641  0.738081
Epoch[12]0: 0:23 0.898702  0.831304 0.466033  0.723945
Epoch[13]0: 0:24 0.903437  0.833740 0.445750  0.711812
Epoch[14]0: 0:26 0.907133  0.835584 0.427372  0.701280
Epoch[15]0: 0:28 0.910440  0.836703 0.410589  0.692051
Epoch[16]0: 0:30 0.913683  0.837954 0.395160  0.683899
Epoch[17]0: 0:31 0.916082  0.838547 0.380898  0.676650
Epoch[18]0

In [135]:
with tf.Session(graph = graph) as session:
    tf.global_variables_initializer()
    saver = tf.train.Saver()
    load_path = saver.restore(session, model_path)
    [p] = session.run([y_class], feed_dict={x_input: testing_data})
import pandas as pd

# Create a Pandas dataframe from some data.
df = pd.DataFrame({'Epoch': epoch_list, 'Time': time_list, 'TrainAcc': train_accuracy_list, 'ValidAcc' : valid_accuracy_list, 'TrainLoss': train_loss_list, 'ValidLoss' : valid_loss_list})

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(model_path + 'Loss.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
df.to_excel(writer, sheet_name='Sheet1')

# Close the Pandas Excel writer and output the Excel file.
writer.save()